In [10]:
import pandas as pd

In [1]:
execution_role = "arn:aws:iam::014257795134:role/ds-mlops-stepfunction-role" # Step function IAM role ARN
baseconstloc='s3://ds-mlops-s3/customonitor/constraints.json'
basestatloc='s3://ds-mlops-s3/customonitor/customonitor/statistics.json'
EndpointName='ds-mlops-xgboost-endpoint'
monitorop='s3://ds-mlops-s3/customonitor/monitoring/monitoring/reports'
XGB_schedular_name='custom-pricing-real-time-monitor-pipeline-scheduler'
v_region = 'us-east-1' # AWS region
sec_groups = ["sg-01d629a900f9b4d92"]
subnets = ["subnet-07bd1dfe6aee76227",
           "subnet-076950ecc89d4340b",
           "subnet-0c5a462cb45a14bab"]
config_bucket = "s3://ds-mlops-s3/customonitor/"
schedule_expression="cron(0 * ? * * *)"

In [2]:
import boto3

client = boto3.client('sagemaker')
sec_groups = ["sg-01d629a900f9b4d92"]
subnets = ["subnet-07bd1dfe6aee76227",
           "subnet-076950ecc89d4340b",
           "subnet-0c5a462cb45a14bab"]

In [1]:
response = client.create_monitoring_schedule(
          MonitoringScheduleName=XGB_schedular_name,#
          MonitoringScheduleConfig={
              'ScheduleConfig': {
                  'ScheduleExpression': schedule_expression
              },
              'MonitoringJobDefinition': {
                  'BaselineConfig': {
                      'ConstraintsResource': {
                          'S3Uri': baseconstloc
                      },
                      'StatisticsResource': {
                          'S3Uri': basestatloc
                      }
                  },
                  'MonitoringInputs': [
                      {
                                  "EndpointInput": {
                                      "EndpointName": EndpointName,
                                      "LocalPath": "/opt/ml/processing/input/monitoring_dataset_input",
                                      "S3InputMode": "File",
                                      "S3DataDistributionType": "FullyReplicated",
                                  }
                              },
                  ],
                  'MonitoringOutputConfig': {
                      'MonitoringOutputs': [
                          {
                              "S3Output": {
                                  "S3Uri": monitorop,
                                  "LocalPath": "/opt/ml/processing/output",
                                  "S3UploadMode": "EndOfJob",
                              }
                          },
                      ],
                  },
                  'MonitoringResources': {
                      "ClusterConfig": {
                          "InstanceCount": 1,
                          "InstanceType": "ml.m5.xlarge",
                          "VolumeSizeInGB": 100,
                      }
                  },
                  'MonitoringAppSpecification': {
                      "ImageUri": "156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer",
                  },
                  'StoppingCondition': {
                      'MaxRuntimeInSeconds': 3600
                  },
                  'NetworkConfig': {
                      'VpcConfig': {
                          "SecurityGroupIds": sec_groups,
                          "Subnets": subnets
                      }
                  },
                  'RoleArn': execution_role
              }
          }
        )


## Clean-up steps

https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html

In [ ]:
# import boto3
# import botocore
# client = boto3.client('sagemaker')
# #client.list_monitoring_executions()
# client.list_monitoring_schedules()

In [ ]:
# client.delete_monitoring_schedule(MonitoringScheduleName= 'custom-pricing-real-time-monitor-pipeline-scheduler')

In [ ]:
# Clean up end point
# client = boto3.client("sagemaker", region_name=region)
# response=client.delete_endpoint(EndpointName=endpoint_name)